# Check Results
This notebook I will use to save each cost type, each agency correction, and the final numbers to a .csv. These .csv's we will use for visualizing results and checking code. The code here will also be the basis of my test code for refactoring.

## Set up Environment

In [44]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [45]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import pandas as pd
import os
import sys
import numpy as np
from sodapy import Socrata

In [46]:
sys.path.insert(0, os.path.abspath("../"))

In [47]:
from Initialize_Agencies import get_agencies
yr = list(range(2016,2020))
agencies = get_agencies(yr)

/Users/alexanderweinstein/Documents/Harris/Summer2020/Carceral_Budgeting/Exploratory/Agency_Classes/State_Agency.py:70: DtypeWarning: Columns (16) have mixed types.Specify dtype option on import or set low_memory=False.
  self.operating_costs = None  # This is where cost type "operating costs" will be stored
/Users/alexanderweinstein/Documents/Harris/Summer2020/Carceral_Budgeting/Exploratory/Agency_Classes/State_Agency.py:70: DtypeWarning: Columns (15,16) have mixed types.Specify dtype option on import or set low_memory=False.
  self.operating_costs = None  # This is where cost type "operating costs" will be stored


In [48]:
final_results = pd.read_csv("Final_by_Agency_Type.csv", index_col = 0)
final_results.rename(columns = {x:int(x) for x in final_results.columns[2:]}, inplace=True)

# Check

In [49]:
index_attr_dict = {" Payroll Costs": "payroll_by_year",
                   " Non-Payroll Operating Costs": "non_payroll_operating_expenditures_by_year",
                   " Pension Costs": "pensions",
                   " Fringe Benefit Costs": "fringe",
                   " Capital Costs": "capital_expenditures_by_year"}
for _, agency in agencies.items():
    for cti, cta in index_attr_dict.items():
        true = final_results.loc[agency.alias + cti , yr].astype(float).values
        current = agency.correction_function(agency.__dict__[cta]).astype(float).values 
        assert np.allclose(a=true, b=current, rtol=1e-10), \
             cti + " for "+ agency.alias+ " not equal. Correct number is " + str(true) + \
        " . New number is " + str(current)
print("all passed!!")

AssertionError:  Payroll Costs for trial_court not equal. Correct number is [23942820.4987764  22913410.46349505 25791232.23978819 28234843.17476416] . New number is [23942820.4987764  22913410.46349505 25791232.23978819 26569925.65739774]

Note that this doesn't check local pensions for trial court